## Complete example for an autotrader
You can use this notebook daily to submit latest Predicto Forecasts to Alpaca, based on your own filters.
(https://predic.to) (https://alpaca.markets)


### For Predicto authentication
To use predicto api and reproduce this notebook, you'll need to have a valid Predicto account. 
If you don't have one, you can create one here: https://predic.to and get a free trial period.

To authenticate you'll need an api session id. To retrieve it, login to https://predic.to and head to your [settings page](https://predic.to/account). Then paste it in the `api_session_id` variable below.

If you get any exception/error while running below code, please make sure your api session id is correct and your subscription/trial is not expired.
Please note that there is a limit to the number of requests you can make per minute, depending on your account type.

### For Alpaca authentication
You'll need an alpaca.markets account. Then you can retrieve your API Key ID and Endpoint from your account page. You can use either paper money or real money. We recommend to experiment with a paper money account first.

More info about Alpaca trade api can be found here: https://github.com/alpacahq/alpaca-trade-api-python/

### Import needed packages

In [1]:
import sys
sys.path.append("../predicto_api/")

import pandas as pd

from predicto_api_wrapper import PredictoApiWrapper, TradeAction, TradeOrderType
from alpaca_api_wrapper import AlpacaApiWrapper

### Prepare and initialize our Alpaca API wrapper
You'll need to have a valid Alpaca account as mentioned above, and replace below variables with your own credentials.

In [2]:
alpaca_api_endpoint = 'https://paper-api.alpaca.markets' # use paper money endpoint for now (test env)
alpaca_api_key_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
alpaca_api_secret_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

alpaca_wrapper = AlpacaApiWrapper(alpaca_api_endpoint, alpaca_api_key_id, alpaca_api_secret_key)

### Prepare and initialize our Predicto wrapper
You'll need to have a valid Predicto account as mentioned above, and get an api session id. Then replace the `api_session_id` variable below

In [3]:
predicto_api_session_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
predicto_api_wrapper = PredictoApiWrapper(predicto_api_session_id)
predicto_api_wrapper.set_alpaca_api_wrapper(alpaca_wrapper)

### Make sure our alpaca keys work as expected
Retrieve some latest prices

In [4]:
print('Latest price of AAPL is: ', alpaca_wrapper.get_latest_price('AAPL'))
print('Latest price of FB   is: ', alpaca_wrapper.get_latest_price('FB'))
print('Latest price of MSFT is: ', alpaca_wrapper.get_latest_price('MSFT'))
print('Latest price of NIO  is: ', alpaca_wrapper.get_latest_price('NIO'))
print('Latest price of TSLA is: ', alpaca_wrapper.get_latest_price('TSLA'))
print('Latest price of V    is: ', alpaca_wrapper.get_latest_price('V'))

print('Looks good \N{grinning face}')

Latest price of AAPL is:  124.46
Latest price of FB   is:  270.11
Latest price of MSFT is:  218.75
Latest price of NIO  is:  47.13
Latest price of TSLA is:  654.24
Latest price of V    is:  206.45
Looks good 😀


### Let's make sure our Predicto Key works as expected
Retrieve supported stocks 

In [5]:
# Get all supported stocks
stocks_json = predicto_api_wrapper.get_supported_tickers()
stocks_df = pd.DataFrame(stocks_json)

# print some information
print('Total tickers supported: {0}'.format(len(stocks_df)))
print('Here is a sample:')
stocks_df.head(15)

Total tickers supported: 44
Here is a sample:


,RelatedStock,RelatedCompany
0,AMD,AMD
1,ATVI,Activision
2,BABA,Alibaba
3,AMZN,Amazon
4,AAL,American Airlines
5,AAPL,Apple
6,BIDU,Baidu
7,BAC,Bank of America
8,BKNG,Booking Holdings
9,CVX,Chevron


# We are ready! 
# Let's run our autotrader
Below call will get latest predicto picks and will submit the ones that match your criteria!

## You can run this daily just after market open (9.31am E.T.)
Feel free to re-adjust below parameters to match your criteria

    abs_change_pct_threshold : the absolute percentage expected change threshold, default is 0.02 (2%)
    actions                  : array with actions filter, default is [int(TradeAction.Buy), int(TradeAction.Sell)]
    average_uncertainty      : threshold for average uncertainty of forecast - the higher the riskier, default 0.15 (15%)
    model_avg_roi            : threshold for the historical average ROI for all the Trade Picks from the stock's model, default 0.0 (non negative ROI)
    symbols                  : array with symbols to trade, if None all of them will be considered
    investment_per_trade     : how much money to use per trade (note we'll submit an order for as many stocks as possible up to this number. If it's not enough for a single stock we'll skip)
    trade_order_type         : TradeOrderType.Bracket or TradeOrderType.TrailingStop. Bracket will set fixed stop loss and take profit prices. TrailingStop will set a trailing stop price.

In [6]:
predicto_api_wrapper.submit_latest_trade_picks(
                                        abs_change_pct_threshold = 0.02,
                                        actions = [int(TradeAction.Buy), int(TradeAction.Sell)],
                                        average_uncertainty = 0.15,
                                        model_avg_roi = 0.0,
                                        symbols = None,
                                        investment_per_trade=1000,
                                        trade_order_type=TradeOrderType.Bracket)

Processing Predicto Trade Picks from 2020-12-20
- More details http://predic.to/exploreroi?minroi=0&sdate=2020-12-20

Using filters
	 abs_change_pct_threshold : 0.02
	 average_uncertainty      : 0.15
	 model_avg_roi            : 0.00
	 symbols                  : All
	 actions                  : ['Buy', 'Sell']

Processing 1/44, Symbol: AMD
	Skipping, didnt match criteria

Processing 2/44, Symbol: ATVI
	Skipping, didnt match criteria

Processing 3/44, Symbol: BABA
	Matched Expected change : -2.45% !
	        Trade Action    : Sell !
	        Avg Uncertainty : 5.81% !
	        Avg ROI of model: 0.62% !

> Alpaca: Submitting Bracket order...
------------------------
Expected price         : 260.0
Actual   price         : 258.22
Original stopLossPrice : 261.594283171
New stopLossPrice      : 259.803368463137
Target Price           : 253.6228673158
Success: submit_order ID: 4267ea0c-47e6-486a-bc2f-e369a452ba5b
------------------------

Processing 4/44, Symbol: AMZN
	Matched Expected change 

# We succesfully submitted our orders !
Those orders will be closed once their take-profit or stop-loss price we set are hit, depending on the TradeOrderType used.

You can inspect your oders at https://app.alpaca.markets/paper/dashboard/orders

You can schedule this script to run daily before market open.

Make sure you understand the risks if you are using real money!